# Traveling Salesman Problem (TSP): Context + Two Implementations

**What you'll find here**
1. Background and context on the Traveling Salesman Problem (TSP)
2. A **dependency-free** TSP solver in pure Python using **2-opt local search**
3. A **Mixed-Integer Linear Programming (MILP)** formulation using **PuLP** (with MTZ subtour elimination)

**How to use this notebook**
- Run Section 2 for a no-dependency solution that works anywhere Python runs.
- Run Section 3 if you have (or can install) `pulp` to see an exact MILP approach for small-to-medium instances.


## 1) Background: What is the TSP?
The **Traveling Salesman Problem (TSP)** asks: given a set of cities and distances between them, find the **shortest possible tour** that visits each city **exactly once** and returns to the starting city.

**Why it's important**
- It captures core ideas in **combinatorial optimization**.
- It's **NP-hard**, so exact solutions scale poorly; heuristics and metaheuristics are key in practice.
- TSP appears in logistics, routing drones/robots, circuit board design, DNA sequencing (as a model), and more.

**Growth of possibilities**
For `n` cities, the number of distinct tours is roughly `(n-1)!/2` (for undirected symmetric distances). Even `n=20` is astronomically large, so brute force is infeasible.

**Two families of approaches**
- **Exact**: MILP/ILP with branch & bound, cutting planes, dynamic programming (e.g., Held–Karp). Guarantees optimality but limited scale.
- **Heuristics/metaheuristics**: nearest neighbor, 2-opt/3-opt, simulated annealing, genetic algorithms, ant colony, etc. Scales better; no optimality guarantee.


## 2) Dependency-free TSP (2-opt local search)
Below is a **single pure-Python** TSP implementation:
- Builds a small set of 2D points
- Computes a distance matrix
- Constructs an initial tour via **nearest neighbor**
- Improves the tour with **2-opt** until no improvements remain

This is great for teaching and small demos (tens to hundreds of cities).

In [1]:
# --- Dependency-free TSP with 2-opt ---
from typing import List, Tuple
import math, random

Point = Tuple[float, float]

def euclid(p: Point, q: Point) -> float:
    return math.hypot(p[0]-q[0], p[1]-q[1])

def build_dist_matrix(pts: List[Point]) -> List[List[float]]:
    n = len(pts)
    D = [[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            d = euclid(pts[i], pts[j])
            D[i][j] = D[j][i] = d
    return D

def tour_length(tour: List[int], D: List[List[float]]) -> float:
    n = len(tour)
    s = 0.0
    for k in range(n):
        i = tour[k]
        j = tour[(k+1) % n]
        s += D[i][j]
    return s

def nearest_neighbor(D: List[List[float]], start: int = 0) -> List[int]:
    n = len(D)
    unv = set(range(n))
    tour = [start]
    unv.remove(start)
    cur = start
    while unv:
        nxt = min(unv, key=lambda j: D[cur][j])
        tour.append(nxt)
        unv.remove(nxt)
        cur = nxt
    return tour

def two_opt_once(tour: List[int], D: List[List[float]]):
    n = len(tour)
    best_gain, best = 0.0, None
    for i in range(n-1):
        for j in range(i+2, n if i>0 else n-1):
            a, b = tour[i], tour[(i+1)%n]
            c, d = tour[j], tour[(j+1)%n]
            old = D[a][b] + D[c][d]
            new = D[a][c] + D[b][d]
            gain = old - new
            if gain > 1e-12 and gain > best_gain:
                best_gain = gain
                best = (i, j)
    if best is not None:
        i, j = best[0]+1, best[1]
        tour[i:j+1] = reversed(tour[i:j+1])
        return tour, True
    return tour, False

def two_opt(tour: List[int], D: List[List[float]]):
    improved = True
    while improved:
        tour, improved = two_opt_once(tour, D)
    return tour

def solve_tsp_dependency_free(points: List[Point], seed: int = 42, restarts: int = 20):
    random.seed(seed)
    D = build_dist_matrix(points)
    n = len(points)
    best_tour, best_len = None, float('inf')
    # a few NN starts
    for s in range(min(n, 5)):
        t0 = nearest_neighbor(D, s)
        t1 = two_opt(t0[:], D)
        L = tour_length(t1, D)
        if L < best_len:
            best_tour, best_len = t1[:], L
    # random restarts
    for _ in range(restarts):
        t0 = list(range(n))
        random.shuffle(t0)
        t1 = two_opt(t0, D)
        L = tour_length(t1, D)
        if L < best_len:
            best_tour, best_len = t1[:], L
    return best_tour, best_len, D

# Example points (edit freely)
CITIES = [
    (0,0),(1,5),(5,2),(6,6),(8,3),(2,1),(7,8),(3,7),(9,4),(4,3)
]

tour, length, D = solve_tsp_dependency_free(CITIES, restarts=50)
print('Best tour (indices):', tour)
print('Best length:', round(length, 4))
print('Readable route:')
for idx in tour:
    print(f'  {idx}: {CITIES[idx]}')
print('  back to start:', CITIES[tour[0]])

Best tour (indices): [2, 9, 5, 0, 1, 7, 3, 6, 8, 4]
Best length: 28.8531
Readable route:
  2: (5, 2)
  9: (4, 3)
  5: (2, 1)
  0: (0, 0)
  1: (1, 5)
  7: (3, 7)
  3: (6, 6)
  6: (7, 8)
  8: (9, 4)
  4: (8, 3)
  back to start: (5, 2)


## 3) Exact MILP with PuLP (MTZ subtour elimination)
The MILP model below solves the **symmetric TSP** exactly for small/medium `n` by adding **Miller–Tucker–Zemlin (MTZ)** constraints.

**Variables**
- `x[i,j] ∈ {0,1}` selects edge (i,j) (undirected handling via `i<j` or symmetry)
- `u[i]` (MTZ ordering vars) break subtours

**Constraints**
- Degree-2 for each node (enter and leave once)
- MTZ: `u[i] - u[j] + n * x[i,j] ≤ n-1` for `i ≠ j`, `i,j ≥ 1`, with `u[0]` fixed

**Note**: You may need to `pip install pulp` in your environment first.

In [2]:
# If PuLP is not installed locally, uncomment the next line:
# !pip install pulp

In [1]:
from typing import List, Tuple
import math
try:
    import pulp
except ImportError:
    raise RuntimeError('PuLP is not installed. Run "!pip install pulp" in a notebook cell or install it in your environment.')

Point = Tuple[float, float]

def euclid(p: Point, q: Point) -> float:
    return math.hypot(p[0]-q[0], p[1]-q[1])

def tsp_mtz_pulp(points: List[Point]):
    n = len(points)
    D = [[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                D[i][j] = euclid(points[i], points[j])

    prob = pulp.LpProblem('TSP_MTZ', pulp.LpMinimize)

    x = {(i,j): pulp.LpVariable(f'x_{i}_{j}', 0, 1, cat='Binary')
         for i in range(n) for j in range(i+1, n)}

    u = {i: pulp.LpVariable(f'u_{i}', lowBound=0, upBound=n-1, cat='Continuous') for i in range(n)}
    prob += (u[0] == 0)

    prob += pulp.lpSum(D[i][j]*x[(i,j)] for i in range(n) for j in range(i+1, n))

    for k in range(n):
        incident = []
        for i in range(n):
            if i < k:
                incident.append(x[(i,k)])
            elif i > k:
                incident.append(x[(k,i)])
        prob += (pulp.lpSum(incident) == 2)

    for i in range(1, n):
        for j in range(1, n):
            if i == j:
                continue
            ij = (i,j) if i<j else (j,i)
            prob += (u[i] - u[j] + (n)*x[ij] <= n-1)

    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    status = pulp.LpStatus[prob.status]
    obj = pulp.value(prob.objective)

    edges = [(i,j) for (i,j), var in x.items() if var.varValue is not None and var.varValue > 0.5]
    adj = {k: [] for k in range(n)}
    for i,j in edges:
        adj[i].append(j)
        adj[j].append(i)
    tour = [0]
    cur, prev = 0, None
    for _ in range(n-1):
        nxt = [v for v in adj[cur] if v != prev][0]
        tour.append(nxt)
        prev, cur = cur, nxt
    return status, obj, tour

CITIES_MTZ = [
    (0,0),(1,5),(5,2),(6,6),(8,3),(2,1),(7,8),(3,7),(9,4),(4,3)
]
status, obj, tour = tsp_mtz_pulp(CITIES_MTZ)
print('Status:', status)
print('Optimal length:', round(obj, 4))
print('Tour (start at 0):', tour + [tour[0]] if tour else tour)

RuntimeError: PuLP is not installed. Run "!pip install pulp" in a notebook cell or install it in your environment.